In [1]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
import os

os.chdir(proj_root())

In [2]:
from coeditor.retrieval_model import RetrievalEditorModel
from coeditor.api import EditPredictionService, BasicQueryEditEncoder, BatchArgs

model_path = get_model_dir(False) / "coeditor-large-query-basic/checkpoint-221507"
model = RetrievalEditorModel.load(model_path)
model.to("cuda:1")
service = EditPredictionService(
    proj_root(),
    model,
    encoder=BasicQueryEditEncoder(max_ref_tks=512),
    batch_args=BatchArgs(
        max_ref_tks=50 * 512, max_ref_dropout=0.0, shuffle_extra_ids=False
    ),
)


/home/jiayi/Projects/SPOT/src/coeditor/retrieval_model.py:255: UserWarning: No extra args found, using default model setting.
  warnings.warn("No extra args found, using default model setting.")


In [6]:
service.suggest_edit(Path("src/coeditor/api.py"), 250)
# service.suggest_edit(Path("src/spot/utils.py"), (218, 0))

num batches: 1
Batch stats: {'mean': '1.0', 'median': '1.0', 'min': '1.0', 'max': '1.0'}
========== Predicted code change ==========
* Modified: 
    ... 12 lines omitted ...
 
                if not file.is_absolute():
                    file = project / file
 
                with timed("get target element"):
                    mname = PythonProject.rel_path_to_module_name(file.relative_to(project))
    -               if (mod := self.now_cache.get(mname, str(os.stat(file).st_mtime))) is None:
    +               if (
    +                   mod := self.parse_cache.get(mname, str(os.stat(file).st_mtime))
    +               ) is None:
                        mod = PythonModule.from_cst(cst.parse_module(file.read_text()), mname)
                    elem = get_elem_by_line(mod, line)
    -           if elem is None:
    -               raise ValueError(f"No code element found at line {line} in file {file}.")
    -           if not isinstance(elem, PythonFunction):
    -              

In [4]:
service.tlogger.as_dataframe()


,name,count,avg_time,total_time
3,run model,1,0.924438,0.924438
1,construct project edit,1,0.691362,0.691362
0,get target element,1,0.429420,0.429420
2,encode edits,1,0.037939,0.037939
